<a href="https://colab.research.google.com/github/VinaySingh561/LARGE-LANGUAGE-MODELS/blob/main/Inp_op_targets_using_Python_Loader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 10.0 MB/s eta 0:00:00


In [2]:
import tiktoken
tokenizer = tiktoken.get_encoding("gpt2")

In [4]:
with open("the-verdict.txt","r", encoding = "utf-8") as f:
  raw_text = f.read()

enc_text = tokenizer.encode(raw_text)
print(len(enc_text))


5145


In [5]:
enc_sample = enc_text[:50]

In [6]:
context_size = 4
x = enc_sample[:context_size]
y = enc_sample[1:context_size+1]
print(f"x: {x}")
print(f"y: {y}")

x: [40, 367, 2885, 1464]
y: [367, 2885, 1464, 1807]


In [10]:
for i in range(1,context_size+1):
  context = enc_sample[:i]
  desire = enc_sample[i]

  print(context, "----->", desire)

[40] -----> 367
[40, 367] -----> 2885
[40, 367, 2885] -----> 1464
[40, 367, 2885, 1464] -----> 1807


In [19]:
for i in range(1,context_size+1):
  context = enc_sample[:i]
  desire = enc_sample[i]

  print(tokenizer.decode(context), "----->", tokenizer.decode([desire]))

I ----->  H
I H -----> AD
I HAD ----->  always
I HAD always ----->  thought


*Data Loader*

In [20]:
import torch
from torch.utils.data import Dataset, DataLoader

class GPTDatasetsV1(Dataset):
  def __init__(self,txt,tokenizer, max_length,stride):
    self.input_ids = []
    self.target_ids = []

    token_ids = tokenizer.encode(txt,allowed_special={"<|endoftext|>"})

    for i in range(0,len(token_ids)-max_length,stride):
      input_chunk = token_ids[i:i+max_length]
      target_chunk = token_ids[i+1:i+max_length+1]
      self.input_ids.append(torch.tensor(input_chunk))
      self.target_ids.append(torch.tensor(target_chunk))

  def __len__(self):
    return len(self.input_ids)

  def __getitem__(self,idx):
    return self.input_ids[idx],self.target_ids[idx]

In [21]:
def create_dataloader_v1(txt,batch_size=4,max_length=256,stride = 128,
                         shuffle = True,drop_last = True,num_workers = 0):
  tokenizer = tiktoken.get_encoding("gpt2")

  # create dataset
  dataset  = GPTDatasetsV1(txt,tokenizer,max_length,stride)

  # create dataloader
  dataloader = DataLoader(
      dataset,
      batch_size=batch_size,
      shuffle=shuffle,
      drop_last=drop_last,
      num_workers=num_workers

  )

  return dataloader

In [22]:
dataloader = create_dataloader_v1(
    raw_text,batch_size=1,max_length=4,stride=1,shuffle=False
)

data_iter = iter(dataloader)
first_batch = next(data_iter)
print(first_batch)

[tensor([[  40,  367, 2885, 1464]]), tensor([[ 367, 2885, 1464, 1807]])]


In [23]:
second_batch = next(data_iter)
print(second_batch)

[tensor([[ 367, 2885, 1464, 1807]]), tensor([[2885, 1464, 1807, 3619]])]


In [30]:
## inncreasing the batch_size
dataloader = create_dataloader_v1(
    raw_text,batch_size=4,max_length=4,stride=4,shuffle=False
)

data_iter = iter(dataloader)
first_batch = next(data_iter)
print("Input  : ", first_batch[0])
print("Target : ",first_batch[1])

Input  :  tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257]])
Target :  tensor([[  367,  2885,  1464,  1807],
        [ 3619,   402,   271, 10899],
        [ 2138,   257,  7026, 15632],
        [  438,  2016,   257,   922]])
